## The imports

In [1]:
import zarr
import xarray as xr
import time
import dask
import glob

# The workers

In [2]:
ask_workers=20
memory='180GB'

In [3]:
from dask_jobqueue import PBSCluster
from dask.distributed import Client
import dask.dataframe as dd

cluster = PBSCluster(cores=1, memory=memory, project='PerfTestPangeo', walltime='04:00:00')
cluster.scale(ask_workers)

c = Client(cluster)

c


Client Scheduler: tcp://10.120.43.22:40605 Dashboard: http://10.120.43.22:8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [4]:
from dask.utils import ensure_dict, format_bytes
    
wk = c.scheduler_info()["workers"]

text="Workers= " + str(len(wk))
memory = [w["memory_limit"] for w in wk.values()]
if all(memory):
    text += ", Memory=" + format_bytes(sum(memory))
print(text)


Workers= 20, Memory=3.60 TB


# The data

In [5]:
files=sorted(glob.glob('/work/ALT/odatis/eNATL60/BLBT02/gridT-2D/eNATL60*gridT-2D*.nc'))
drop_vars = ['nav_lat', 'nav_lon', 'somxl010', 'sosaline', 'sosstsst']
extra_coord_vars = []
chunks = dict(time_counter=240, y=240, x=480)
open_kwargs = dict(drop_variables=(drop_vars + extra_coord_vars),
                   chunks=chunks,decode_cf=True,decode_times=False,
                   concat_dim="time_counter",combine='nested')

In [8]:
%time ds = xr.open_mfdataset(files, parallel=True, **open_kwargs)

CPU times: user 3.75 s, sys: 197 ms, total: 3.95 s
Wall time: 4.77 s


In [9]:
%time mean=ds.sossheig.mean(dim='time_counter')

CPU times: user 1.9 s, sys: 19.4 ms, total: 1.92 s
Wall time: 1.93 s


In [ ]:
%time mean.load()

In [ ]:
cluster.close()